In [1]:
import sqlite3
import pymssql
import pandas as pd

ruta_archivo1 = 'C:\\Users\\valen\\OneDrive\\Documentos\\VALEN UAO\\ETI\\ACTIVIDAD1\\BASE_DE_TRABAJO_SABER11.xlsx'

df1=pd.read_excel(ruta_archivo1)
ruta_archivo2 = 'C:\\Users\\valen\\OneDrive\\Documentos\\VALEN UAO\\ETI\\ACTIVIDAD1\\COMPLETA-CARACTERIZACIÓN EST PRIMER SEMESTRE (1) - copia.xlsx'

df2=pd.read_excel(ruta_archivo2)


In [2]:
import pandas as pd
from unidecode import unidecode
# Función para remover tildes de los nombres de las columnas
def remover_tildes_columnas(df):
    df.columns = [unidecode(col) for col in df.columns]
    return df

# Aplicar la función a ambas bases de datos
df1 = remover_tildes_columnas(df1)
df2 = remover_tildes_columnas(df2)

In [3]:
# Asegurar que la columna clave tenga el mismo nombre en ambas bases
df1.rename(columns={"Codigo Estudiante": "codigo_estudiante"}, inplace=True)
df2.rename(columns={"Codigo Estudiante": "codigo_estudiante"}, inplace=True)

In [4]:
# Completar df1 con la información de df2 usando "codigo_estudiante" como clave
# Se mantiene toda la información de df1 y se agregan datos coincidentes de df2
df_final = pd.merge(df1, df2, on="codigo_estudiante", how="left")

In [5]:
from unidecode import unidecode

# Normalizar nombres de columnas en df_final sin corchetes
df_final.columns = [
    unidecode(col).replace(' ', '_').replace('-', '_').replace('.', '_')
    for col in df_final.columns
]


In [6]:
from unidecode import unidecode

# Normalizar nombres de columnas en df_final sin corchetes
df_final.columns = [unidecode(col).replace(" ", "_") for col in df_final.columns]

# Crear la estructura de la tabla con los tipos de datos (sin corchetes `[]`)
columnas_sql = ", ".join([
    f"{col}" if df_final[col].dtype == "O" else f"{col} FLOAT"
    for col in df_final.columns
])

In [7]:
import pymssql

#conexión con la BD
conn = pymssql.connect(
    server='127.0.0.1',
    user='sa',
    password='12345678',
    #database='master1',
    as_dict= False
)

# Se crea el cursor para manejo de qwery
cur = conn.cursor()

In [8]:
# Crear una nueva base de datos en SQL Server
db_name = "Actividad_ETL"

try:
    # Se verifica que no haya transacciones activas en la base de datos
    cur.execute("IF @@TRANCOUNT > 0 ROLLBACK TRANSACTION")

    # Se consulta si la base de datos ya existe en el sistema
    cur.execute(f"SELECT name FROM sys.databases WHERE name = '{db_name}'")
    
    # Se obtiene el resultado de la consulta
    resultado = cur.fetchone()

    # Si la base de datos ya existe, se muestra un mensaje
    if resultado:
        print(f"La base de datos '{db_name}' ya existe.")
    else:
        # Si la base de datos no existe, se crea con CREATE DATABASE
        cur.execute(f"CREATE DATABASE {db_name}")
        print(f"Base de datos '{db_name}' creada.")

# Captura cualquier error que ocurra durante la ejecución del código
except Exception as e:
    print(f"Error al verificar o crear la base de datos: {e}")

Base de datos 'Actividad_ETL' creada.


In [13]:
import re

# Nombre de la nueva tabla
new_tb_name = "tabla_etl_nueva1"

try:
    # Función para limpiar nombres de columnas y evitar errores de SQL
    def limpiar_nombre(col):
        col = col.strip().replace(' ', '_')  # Reemplazar espacios por guion bajo
        col = re.sub(r'[^a-zA-Z0-9_]', '', col)  # Eliminar caracteres especiales
        return col[:128]  # Limitar a 128 caracteres máximo

    # Construcción de la definición de columnas SQL
    columnas_sql = ", ".join([
        f"[{limpiar_nombre(col)}] NVARCHAR(MAX)" if df_final[col].dtype == "O" 
        else f"[{limpiar_nombre(col)}] FLOAT"
        for col in df_final.columns
    ])

    # Query para crear la tabla si no existe
    sql_query = f"""
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES 
                   WHERE TABLE_NAME = '{new_tb_name}' AND TABLE_SCHEMA = 'dbo') 
    BEGIN 
        CREATE TABLE dbo.{new_tb_name} ({columnas_sql}) 
    END
    """

    cur.execute(sql_query)

    print(f"Tabla '{new_tb_name}' creada exitosamente en la base de datos 'Actividad_ETL'.")

except Exception as e:
    print(f"Error al crear la tabla: {e}")

Tabla 'tabla_etl_nueva1' creada exitosamente en la base de datos 'Actividad_ETL'.


In [15]:
import pymssql

#conexión con la BD
conn = pymssql.connect(
    server='127.0.0.1',
    user='sa',
    password='12345678',
    database='Actividades_ETL',
    as_dict= False
)


import numpy as np

try:
    # 1. Verificar si la tabla existe en la base de datos
    cur.execute(f"SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'tabla_etl_nueva1'")
    existe = cur.fetchone()[0]

    if existe == 0:
        print("La tabla 'tabla_etl_nueva1' no existe en la base de datos.")
    else:
        print(" La tabla 'tabla_etl_nueva1' existe en la base de datos.")

        # 2. Insertar los datos en la tabla
        print(" Insertando datos en la tabla 'tabla_etl_nueva1'...")

        # Reemplazar NaN con None para evitar errores en SQL Server
        df_final_clean = df_final.replace({np.nan: None})

        # Construcción de la consulta de inserción
        columnas = ", ".join([f"[{col.replace(' ', '_')}]" for col in df_final_clean.columns])
        valores_placeholder = ", ".join(["%s"] * len(df_final_clean.columns))

        insert_query = f"INSERT INTO tabla_etl_nueva1 ({columnas}) VALUES ({valores_placeholder})"

        # Convertir DataFrame a lista de tuplas para inserción
        datos_a_insertar = [tuple(row) for row in df_final_clean.itertuples(index=False, name=None)]

        # Insertar datos en lotes
        cur.executemany(insert_query, datos_a_insertar)
        conn.commit()

        print(f" Se han insertado {len(df_final_clean)} filas en la tabla 'tabla_etl_nueva1'.")

except Exception as e:
    conn.rollback()
    print(f" Error al insertar datos en la tabla: {e}")

 La tabla 'tabla_etl_nueva1' existe en la base de datos.
 Insertando datos en la tabla 'tabla_etl_nueva1'...
 Se han insertado 11729 filas en la tabla 'tabla_etl_nueva1'.
